### importing libaries


In [ ]:
import mne
import matplotlib as plt
import os
import numpy as np
from glob import glob
import warnings
from scipy import stats
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler, LabelEncoder




### Data Preparation: Combining Subject Files

Our goal is **person identification** (creating a "brainprint") rather than "task classification." Because of this, our data preparation strategy is different from a typical motor imagery analysis.

#### 1. Ignoring `.event` Files

* **Why:** The `.event` files only contain markers for *what* task the subject was performing (e.g., "imagined left hand," "rest").
* **Our Goal:** For this project, the **label is the person** (e.g., `S001`, `S002`), not the task. We don't care *what* they were doing; we only care *who* they are.
* **Action:** All `.event` files were ignored. We only used the `.edf` (data) files.

#### 2. Combining (Concatenating) `.edf` Files

* **What:** For each subject, we loaded all 14 of their `.edf` files (from `R01` to `R14`).
* **Why:** We want to create one single, long recording for each person. This provides a robust "signature" of their brain activity across various states (resting, task-focused, etc.).
* **Action:** We used `mne.concatenate_raws()` to "stitch" the 14 files together, end-to-end, into one MNE `Raw` object per subject.

#### 3. Resampling to a Common Frequency (128 Hz)

* **Problem:** The dataset is inconsistent. Some files (especially for subjects 89+) were recorded at 160 Hz, while others were recorded at 128 Hz. MNE cannot concatenate files with different sampling frequencies (`sfreq`).
* **Solution:** We resampled *every* file to a common frequency of **128.0 Hz** *before* concatenating them.
* **Action:** This was done using `raw.resample(128.0)` right after loading each `.edf` file.

#### 4. Saving the Combined Files

* **Action:** The final, combined `Raw` object for each subject was saved as a `.fif` file (e.g., `S001_combined_raw.fif`).
* **Why:** The `.fif` format is native to MNE and is much more efficient to load in our future notebooks than re-loading and re-combining all 14 `.edf` files every time.

In [ ]:

base_data_path = r'd:\EEG\1.0.0'
total_subjects = 109
COMMON_SFREQ = 128.0
for i in range(1, total_subjects + 1):
    subject_name = f'S{i:03d}'
    subject_folder = os.path.join(base_data_path, subject_name)   
    run_files = glob(os.path.join(subject_folder, f'{subject_name}R*.edf'))
    run_files.sort()   
    raws_list = []
    print(f"Found {len(run_files)} files. Loading and resampling...")
    for file_path in run_files:
        raw = mne.io.read_raw_edf(file_path, preload=True)
        
        if raw.info['sfreq'] != COMMON_SFREQ:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                raw.resample(COMMON_SFREQ)
        raws_list.append(raw)
    combined_raw = mne.concatenate_raws(raws_list)
    output_filename = os.path.join(subject_folder, f'{subject_name}_combined_raw.fif')
    combined_raw.save(output_filename, overwrite=True)



## Step 2: Apply Preprocessing Pipeline

In this step, we loop through every subject's combined `.fif` file and apply our core preprocessing pipeline.

The pipeline consists of:
1.  **Loading** the combined file (e.g., `S001_combined_raw.fif`).
2.  **Band-pass Filtering:** Keeping only the 1 Hz to 40 Hz frequency range, which contains the most relevant brain activity and removes slow drifts and high-frequency noise.
3.  **Notch Filtering:** Removing the 50 Hz power-line interference.
4.  **Re-referencing:** Setting the reference to the "Common Average Reference" (CAR) to remove widespread noise.
5.  **Saving:** Saving the newly cleaned data to a new file (e.g., `S001_preprocessed_raw.fif`).


In [ ]:
total_subjects = 109
for i in range(1, total_subjects + 1):
    subject_name = f'S{i:03d}'
    file_path = os.path.join(base_data_path, subject_name, f'{subject_name}_combined_raw.fif')
    raw = mne.io.read_raw_fif(file_path, preload=True)
    # Filter
    raw.filter(1.0, 40.0, method='fir', fir_design='firwin')
    # Notch filter
    raw.notch_filter(50.0, method='spectrum_fit')
    # Re-referencing
    raw.set_eeg_reference(ref_channels='average')
    # Save the preprocessed data
    output_path = os.path.join(base_data_path, subject_name, f'{subject_name}_preprocessed_raw.fif')
    raw.save(output_path, overwrite=True)


## Step 3: Epoching the Preprocessed Data

Now that we have 109 clean, continuous files, we need to chop them into small, uniform "samples" (called **epochs**) that we can feed into a Deep learning model.

This code will:
1.  Create a new central directory named `epochs_data` to store all the final epoch files.
2.  Loop through each of the 109 `_preprocessed_raw.fif` files.
3.  Manually "chop" the data into **2-second** long epochs.
4.  Use a **50% overlap** (1 second). This is a data augmentation technique that gives us twice as many epochs (e.g., 0-2s, 1-3s, 2-4s, etc.).
5.  Save these epochs as a NumPy array (`.npy` file) in the `epochs_data` folder.

We are converting our data from the MNE `.fif` format into a pure NumPy array format, which is the standard for most machine learning libraries.

### How it Works

The code uses two functions:

* **`create_fixed_length_epochs(...)`**: This is the "worker" function. It takes a single `raw` file and performs the manual sliding window logic. It extracts a 2-second chunk, moves forward 1 second, extracts the next chunk, and so on.
* **`create_epochs_all_subjects(...)`**: This is the "manager" function. It loops through all 109 subjects, loads the preprocessed file for each one, and calls the "worker" function to create and save the epochs.

In [ ]:

def create_epochs_all_subjects(base_data_path, total_subjects=109):
    """
    Create epochs for all 109 subjects
    """
    epochs_dir = os.path.join(base_data_path, 'epochs_data')
    os.makedirs(epochs_dir, exist_ok=True)
    for i in range(1, total_subjects + 1):
        subject_name = f'S{i:03d}'
        input_path = os.path.join(base_data_path, subject_name, f'{subject_name}_preprocessed_raw.fif')
        output_path = os.path.join(epochs_dir, f'{subject_name}_epochs.npy')     
        raw = mne.io.read_raw_fif(input_path, preload=True)
        epochs = create_fixed_length_epochs(raw)
        np.save(output_path, epochs)

def create_fixed_length_epochs(raw, epoch_duration=2.0, overlap=0.5, sfreq=128.0):
    """
    Create epochs with 2s duration and 50% overlap
    """
    epoch_length = int(epoch_duration * sfreq)
    overlap_samples = int(overlap * epoch_length)
    step_size = epoch_length - overlap_samples
    
    data = raw.get_data()
    n_channels, n_times = data.shape
    
    epochs = []
    start = 0
    
    while start + epoch_length <= n_times:
        epoch = data[:, start:start + epoch_length]
        epochs.append(epoch)
        start += step_size
    
    return np.array(epochs)


create_epochs_all_subjects(base_data_path)

## Step 4: Feature Extraction

This is the final step in our data preparation pipeline. We will convert the 2D epoch data (64 channels x 256 time points) into a 1D "feature vector" that a machine learning model can understand.

This script will:
1.  Create a new directory, `features_data`, to store our final dataset.
2.  Loop through each subject's `_epochs.npy` file.
3.  For each 2-second epoch, it will calculate a list of statistical features.
4.  Crucially, it **creates the labels (`y`)**. For every epoch from subject `S001`, it assigns the label `1`. For `S002`, it assigns `2`, and so on.
5.  It saves two final files:
    * `X_features.npy`: A single, massive 2D array containing all features from all epochs from all subjects.
    * `y_labels.npy`: A single, 1D array containing the corresponding subject label for each epoch.

These two files, `X` and `y`, are the only things we need for training our "brainprint" classifier.

### What Features Are Extracted?

The code extracts **20 features** from *each* of the 64 channels and concatenates them into one long vector.

**20 features/channel * 64 channels = 1280 total features per epoch.**

The 20 features are:

* **10 Time-Domain Features:**
    1.  Mean
    2.  Standard Deviation
    3.  Median
    4.  Minimum
    5.  Maximum
    6.  Peak-to-Peak (Max - Min)
    7.  Mean Absolute Value
    8.  Skewness
    9.  Kurtosis
    10. Root Mean Square (RMS)

* **10 Frequency-Domain Features** (Calculated using Welch's PSD):
    1.  Delta Band Power (0.5-4 Hz)
    2.  Theta Band Power (4-8 Hz)
    3.  Alpha Band Power (8-13 Hz)
    4.  Beta Band Power (13-30 Hz)
    5.  Gamma Band Power (30-40 Hz)
    6.  Mean PSD
    7.  Std Deviation of PSD
    8.  Median PSD
    9.  Dominant Frequency (freq with max power)
    10. Maximum PSD value

In [ ]:
import numpy as np
import os
from scipy import stats
from scipy.signal import welch

def extract_features_all_subjects(base_data_path, total_subjects=109):
    """
    Extract features from epochs for all 109 subjects
    """
    epochs_dir = os.path.join(base_data_path, 'epochs_data')
    features_dir = os.path.join(base_data_path, 'features_data')
    os.makedirs(features_dir, exist_ok=True) 
    all_features = []
    all_labels = []
    
    for i in range(1, total_subjects + 1):
        subject_name = f'S{i:03d}'
        epochs_path = os.path.join(epochs_dir, f'{subject_name}_epochs.npy')
        features_path = os.path.join(features_dir, f'{subject_name}_features.npy')
        labels_path = os.path.join(features_dir, f'{subject_name}_labels.npy')
 
        # Load epochs
        epochs = np.load(epochs_path)
        
        # Extract features
        features = extract_features_from_epochs(epochs)
        
        # Create labels (subject ID)
        labels = np.full(len(features), i)  # Label with subject number
        
        # Save features and labels for this subject
        np.save(features_path, features)
        np.save(labels_path, labels)
        
        # Collect for combined dataset
        all_features.append(features)
        all_labels.append(labels)
    
    # Combine all data
    X = np.vstack(all_features)
    y = np.hstack(all_labels)
    
    # Save combined dataset
    np.save(os.path.join(features_dir, 'X_features.npy'), X)
    np.save(os.path.join(features_dir, 'y_labels.npy'), y)
    
    return X, y

def extract_features_from_epochs(epochs, sfreq=128.0):
    """
    Extract features from epochs (shape: n_epochs × 64 channels × 256 time points)
    """
    features_list = []
    
    for epoch in epochs:
        # epoch shape: (64 channels × 256 time points)
        epoch_features = []
        
        for channel_data in epoch:
            # Extract features for each channel
            channel_features = extract_channel_features(channel_data, sfreq)
            epoch_features.extend(channel_features)
        
        features_list.append(epoch_features)
    
    return np.array(features_list)

def extract_channel_features(channel_data, sfreq=128.0):
    """
    Extract 20 features from a single channel's data
    """
    features = []
    
# Time domain features
    features.extend([
        np.mean(channel_data),                    # 1. Mean
        np.std(channel_data),                     # 2. Standard deviation
        np.median(channel_data),                  # 3. Median
        np.min(channel_data),                     # 4. Minimum
        np.max(channel_data),                     # 5. Maximum
        np.ptp(channel_data),                     # 6. Peak-to-peak
        np.mean(np.abs(channel_data)),            # 7. Mean absolute value
        stats.skew(channel_data),                 # 8. Skewness
        stats.kurtosis(channel_data),             # 9. Kurtosis
        np.sqrt(np.mean(channel_data**2)),        # 10. RMS
    ])
    
# Frequnency domain features
    freqs, psd = welch(channel_data, fs=sfreq, nperseg=min(256, len(channel_data)))
    
    # EEG frequency bands
    bands = {
        'delta': (0.5, 4),
        'theta': (4, 8), 
        'alpha': (8, 13),
        'beta': (13, 30),
        'gamma': (30, 40)
    }
    
    # Band power (11-15)
    for band_name, (low_freq, high_freq) in bands.items():
        band_mask = (freqs >= low_freq) & (freqs <= high_freq)
        if np.any(band_mask):
            band_power = np.sum(psd[band_mask])
            features.append(band_power)
        else:
            features.append(0.0)
    
    # Spectral features (16-20)
    features.extend([
        np.mean(psd),                    # 16. Mean PSD
        np.std(psd),                     # 17. Std PSD
        np.median(psd),                  # 18. Median PSD
        freqs[np.argmax(psd)],           # 19. Dominant frequency
        np.max(psd),                     # 20. Maximum PSD
    ])
    
    return features

# Run feature extraction for all subjects
X, y = extract_features_all_subjects(base_data_path)



NameError: name 'base_data_path' is not defined

## Step 5: Data Aggregation and Normalization

In this step, we aggregate the extracted features from all **109 subjects** into a single dataset to prepare for Deep Learning training.

**Key Operations:**
1. **Data Loading:** Iteratively loads `_features.npy` and `_labels.npy` for subjects `S001` to `S109`.
2. **Concatenation:** Stacks all subject data into a unified feature matrix ($X$) and label vector ($y$).
3. **Feature Scaling:** Applies **Standard Scaling** ($z = \frac{x - \mu}{\sigma}$) to normalize features, ensuring stable model convergence.
4. **Label Encoding:** Converts subject IDs into 0-indexed integers (0 to 108) using `LabelEncoder`.
5. **Persistence:** Saves the normalized datasets and scaler artifacts to the `deep_learning_data/` directory for future inference.

In [ ]:


def combine_and_save_all_features(base_data_path, total_subjects=109):
    """
    Load all 109 feature files, combine them, normalize, and save for DL
    """
    features_dir = os.path.join(base_data_path, 'features_data')
    dl_dir = os.path.join(base_data_path, 'deep_learning_data')
    os.makedirs(dl_dir, exist_ok=True)
    all_features = []
    all_labels = []
    
    # Load all 109 feature files
    for i in range(1, total_subjects + 1):
        subject_name = f'S{i:03d}'
        features_path = os.path.join(features_dir, f'{subject_name}_features.npy')
        labels_path = os.path.join(features_dir, f'{subject_name}_labels.npy')
        
        print(f"Loading {subject_name}...")
        
        # Load features and labels for this subject
        features = np.load(features_path)
        labels = np.load(labels_path)
        
        all_features.append(features)
        all_labels.append(labels)

    # Combine all data
    X = np.vstack(all_features)
    y = np.hstack(all_labels)
    # Feature Scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    # Label Encoding (1-109 → 0-108)
    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(y)
    np.save(os.path.join(dl_dir, 'X_normalized.npy'), X_scaled)
    np.save(os.path.join(dl_dir, 'y_encoded.npy'), y_encoded)
    # Also save the scaler and encoder for future use
    np.save(os.path.join(dl_dir, 'scaler_mean.npy'), scaler.mean_)
    np.save(os.path.join(dl_dir, 'scaler_scale.npy'), scaler.scale_)
    np.save(os.path.join(dl_dir, 'label_encoder_classes.npy'), encoder.classes_)
    
  
    return X_scaled, y_encoded

# Run it
X_normalized, y_encoded = combine_and_save_all_features(base_data_path)

